In [1]:
import pandas as pd

In [2]:
sample_df = pd.read_csv("./sample_data/product_name_sample_data.csv")
sample_df.head()

,product_name,description
0,草本花香洗髮露 600毫升,揉合100%日本培植的有機草本植物，令髮絲彷如重生，令變得柔滑清爽，氛芳花香，令你彷如置身大...
1,Voost 運動水樽,VOOST MUG
2,完美遮瑕筆306 (1.5ml),質地柔亮潤澤，遮瑕的同時去除暗沉，作為 highlight 使用能提亮妝容<BR><BR>獨...
3,高效防脫增生洗髮液 150毫升,ANTI HAIR LOSS SHAMP
4,ISOTONIC 運動水溶片青檸檬味十片裝,幫助人體代謝碳水化合物、脂肪和蛋白質; 快速補充水份與電解質，促進神經肌肉傳導; 有助維持肌...


In [3]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32372 entries, 0 to 32371
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_name  32361 non-null  object
 1   description   31501 non-null  object
dtypes: object(2)
memory usage: 505.9+ KB


In [4]:
sample_df = sample_df.fillna("*")

In [5]:
product_name_df = sample_df[["product_name"]]
description_df = sample_df[["description"]]

product_name_df.columns = ["doc"]
description_df.columns = ["doc"]

long_df = pd.concat([product_name_df, description_df])
long_df.head()

,doc
0,草本花香洗髮露 600毫升
1,Voost 運動水樽
2,完美遮瑕筆306 (1.5ml)
3,高效防脫增生洗髮液 150毫升
4,ISOTONIC 運動水溶片青檸檬味十片裝


In [6]:
long_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64744 entries, 0 to 32371
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   doc     64744 non-null  object
dtypes: object(1)
memory usage: 1011.6+ KB


In [7]:
import re


def clean_text(text:str) -> str:
    # 去除 HTML 標籤
    text = re.sub(r'<.*?>', ',', text)
    # 去除非字母字符以及英文
    text = re.sub(r'[^\u4e00-\u9fa5]', ',', text)
    text = re.sub(r",+", ",", text)

    return text

In [8]:
long_df["doc"] = long_df["doc"].apply(clean_text)
long_df.iloc[:20]

,doc
0,"草本花香洗髮露,毫升"
1,",運動水樽"
2,"完美遮瑕筆,"
3,"高效防脫增生洗髮液,毫升"
4,",運動水溶片青檸檬味十片裝"
5,強生便利貼公主膠布
6,"牛油果滋養修護晚霜,毫升"
7,",麥蘆卡蜂蜜,克,"
8,"光采豐蜜唇釉,"
9,",金盞花面霜"


In [9]:
# def do_n_gram(doc: str, n: int=2) -> list[str]:
#     pattern = re.compile(",")
#     return [doc[i: i+n] for i in range(len(doc) - (n-1)) if not (bool(pattern.search(doc[i: i+n])))]


def do_n_gram(doc: str, n: int=2) -> list[str]:
    return [doc[i: i+n] for i in range(len(doc) - (n-1)) if not ("," in doc[i: i+n])]

In [10]:
long_df["count"] = 1

In [11]:
for i in range(2, 6):
    long_df[f"{i}_gram"] = long_df["doc"].apply(lambda x: do_n_gram(x, i))

In [12]:
long_df.iloc[:20]

,doc,count,2_gram,3_gram,4_gram,5_gram
0,"草本花香洗髮露,毫升",1,"[草本, 本花, 花香, 香洗, 洗髮, 髮露, 毫升]","[草本花, 本花香, 花香洗, 香洗髮, 洗髮露]","[草本花香, 本花香洗, 花香洗髮, 香洗髮露]","[草本花香洗, 本花香洗髮, 花香洗髮露]"
1,",運動水樽",1,"[運動, 動水, 水樽]","[運動水, 動水樽]",[運動水樽],[]
2,"完美遮瑕筆,",1,"[完美, 美遮, 遮瑕, 瑕筆]","[完美遮, 美遮瑕, 遮瑕筆]","[完美遮瑕, 美遮瑕筆]",[完美遮瑕筆]
3,"高效防脫增生洗髮液,毫升",1,"[高效, 效防, 防脫, 脫增, 增生, 生洗, 洗髮, 髮液, 毫升]","[高效防, 效防脫, 防脫增, 脫增生, 增生洗, 生洗髮, 洗髮液]","[高效防脫, 效防脫增, 防脫增生, 脫增生洗, 增生洗髮, 生洗髮液]","[高效防脫增, 效防脫增生, 防脫增生洗, 脫增生洗髮, 增生洗髮液]"
4,",運動水溶片青檸檬味十片裝",1,"[運動, 動水, 水溶, 溶片, 片青, 青檸, 檸檬, 檬味, 味十, 十片, 片裝]","[運動水, 動水溶, 水溶片, 溶片青, 片青檸, 青檸檬, 檸檬味, 檬味十, 味十片, ...","[運動水溶, 動水溶片, 水溶片青, 溶片青檸, 片青檸檬, 青檸檬味, 檸檬味十, 檬味十...","[運動水溶片, 動水溶片青, 水溶片青檸, 溶片青檸檬, 片青檸檬味, 青檸檬味十, 檸檬味..."
5,強生便利貼公主膠布,1,"[強生, 生便, 便利, 利貼, 貼公, 公主, 主膠, 膠布]","[強生便, 生便利, 便利貼, 利貼公, 貼公主, 公主膠, 主膠布]","[強生便利, 生便利貼, 便利貼公, 利貼公主, 貼公主膠, 公主膠布]","[強生便利貼, 生便利貼公, 便利貼公主, 利貼公主膠, 貼公主膠布]"
6,"牛油果滋養修護晚霜,毫升",1,"[牛油, 油果, 果滋, 滋養, 養修, 修護, 護晚, 晚霜, 毫升]","[牛油果, 油果滋, 果滋養, 滋養修, 養修護, 修護晚, 護晚霜]","[牛油果滋, 油果滋養, 果滋養修, 滋養修護, 養修護晚, 修護晚霜]","[牛油果滋養, 油果滋養修, 果滋養修護, 滋養修護晚, 養修護晚霜]"
7,",麥蘆卡蜂蜜,克,",1,"[麥蘆, 蘆卡, 卡蜂, 蜂蜜]","[麥蘆卡, 蘆卡蜂, 卡蜂蜜]","[麥蘆卡蜂, 蘆卡蜂蜜]",[麥蘆卡蜂蜜]
8,"光采豐蜜唇釉,",1,"[光采, 采豐, 豐蜜, 蜜唇, 唇釉]","[光采豐, 采豐蜜, 豐蜜唇, 蜜唇釉]","[光采豐蜜, 采豐蜜唇, 豐蜜唇釉]","[光采豐蜜唇, 采豐蜜唇釉]"
9,",金盞花面霜",1,"[金盞, 盞花, 花面, 面霜]","[金盞花, 盞花面, 花面霜]","[金盞花面, 盞花面霜]",[金盞花面霜]


In [13]:
bigram_df = long_df[["2_gram", "count"]].explode("2_gram").groupby("2_gram", as_index=False).agg({"count": "sum"}).sort_values("count", ascending=False).reset_index(drop=True)
# bigram_df = bigram_df[~bigram_df["2_gram"].str.contains("#")].reset_index(drop=True)
bigram_df

,2_gram,count
0,肌膚,15119
1,配方,6583
2,保濕,5935
3,使用,5884
4,精華,5195
...,...,...
149680,种天,1
149681,种抗,1
149682,种有,1
149683,种水,1


In [14]:
trigram_df = long_df[["3_gram", "count"]].explode("3_gram").groupby("3_gram", as_index=False).agg({"count": "sum"}).sort_values("count", ascending=False).reset_index(drop=True)
# trigram_df = trigram_df[~trigram_df["3_gram"].str.contains("#")].reset_index(drop=True)
trigram_df

,3_gram,count
0,維他命,2812
1,屈臣氏,1828
2,防腐劑,1193
3,型美甲,1179
4,美甲片,1124
...,...,...
343566,杯杯麵,1
343567,杯柔滑,1
343568,杯榨果,1
343569,杯水的,1


In [15]:
fourgram_df = long_df[["4_gram", "count"]].explode("4_gram").groupby("4_gram", as_index=False).agg({"count": "sum"}).sort_values("count", ascending=False).reset_index(drop=True)
# fourgram_df = fourgram_df[~fourgram_df["4_gram"].str.contains("#")].reset_index(drop=True)
fourgram_df

,4_gram,count
0,透明質酸,1111
1,型美甲片,1040
2,膠原蛋白,824
3,指甲品牌,747
4,全新造型,746
...,...,...
427003,士賦顏黃,1
427004,獨家特級,1
427005,士豆澱粉,1
427006,士護膚產,1


In [16]:
fivegram_df = long_df[["5_gram", "count"]].explode("5_gram").groupby("5_gram", as_index=False).agg({"count": "sum"}).sort_values("count", ascending=False).reset_index(drop=True)
# fivegram_df = fivegram_df[~fivegram_df["5_gram"].str.contains("#")].reset_index(drop=True)
fivegram_df

,5_gram,count
0,甲品牌全新,744
1,全新造型美,744
2,指甲品牌全,744
3,牌全新造型,744
4,品牌全新造,744
...,...,...
427764,旅途中快速,1
427765,旅途中使用,1
427766,旅行電水壺,1
427767,旅行野餐籃,1


In [17]:
# def find_matches(ngram1, ngram2):
#     return ngram2[(ngram2.iloc[:, 1] > 1) & (ngram2.iloc[:, 0].str.contains(ngram1))].iloc[:, 0].to_list()

In [18]:
def find_matches(term, ngram):
    def matches_position(term, text):
        pattern = re.compile(term)

        position = pattern.search(text)
        if position:
            return position.span()[0]
        return None
    temp_df = ngram[ngram.iloc[:, 1] > 1].copy()
    temp_df["matches"] = temp_df.iloc[:, 0].map(lambda x: matches_position(term, x))  # 100 → 12.8s
    return {row.iloc[0]: [row.iloc[1], row.iloc[2]] for _, row in temp_df[~(temp_df["matches"].isna())].iterrows()}

In [19]:
import numpy as np
def find_matches_np(term, ngram):  # 100 -> 12.7s
    array = ngram.iloc[:, 0][ngram.iloc[:, 1] > 1].to_list()

    matches = np.char.find(array, term) != -1

    return matches.sum()

In [20]:
bi_tri_df = bigram_df.copy()

count_threshold_2_gram = 300

bi_tri_df["keep_2_gram"] = bi_tri_df["count"].map(lambda x: True if x > count_threshold_2_gram else False)
bi_tri_df

,2_gram,count,keep_2_gram
0,肌膚,15119,True
1,配方,6583,True
2,保濕,5935,True
3,使用,5884,True
4,精華,5195,True
...,...,...,...
149680,种天,1,False
149681,种抗,1,False
149682,种有,1,False
149683,种水,1,False


In [21]:
bi_tri_df["compare"] = bi_tri_df["2_gram"][bi_tri_df["keep_2_gram"] == True].apply(lambda x: find_matches(x, trigram_df))

In [22]:
bi_tri_df

,2_gram,count,keep_2_gram,compare
0,肌膚,15119,True,"{'令肌膚': [916, 1.0], '為肌膚': [767, 1.0], '肌膚的': ..."
1,配方,6583,True,"{'配方奶': [323, 0.0], '水配方': [165, 1.0], '配方含': ..."
2,保濕,5935,True,"{'效保濕': [560, 1.0], '保濕成': [441, 0.0], '潤保濕': ..."
3,使用,5884,True,"{'續使用': [243, 1.0], '使用方': [227, 0.0], '使用後': ..."
4,精華,5195,True,"{'精華液': [395, 0.0], '取精華': [290, 1.0], '濕精華': ..."
...,...,...,...,...
149680,种天,1,False,NaN
149681,种抗,1,False,NaN
149682,种有,1,False,NaN
149683,种水,1,False,NaN


In [23]:
bi_tri_df["group_count"] = bi_tri_df["compare"].apply(lambda x: len(x.items()) if isinstance(x, dict) else x)
bi_tri_df["total_related_3_gram"] = bi_tri_df["compare"].apply(lambda x: np.array(list(x.values()))[:, 0].sum() if isinstance(x, dict) and len(x.values()) > 0 else x)
bi_tri_df["compare"] = bi_tri_df["compare"].map(lambda x: x.items() if isinstance(x, dict) else x)
# bi_tri_df[~(bi_tri_df["compare"].isna())]
bi_tri_df_expand = bi_tri_df.explode("compare")  # .groupby("2_gram", as_index=False)  # .agg({"count": "sum"}).sort_values("count", ascending=False).reset_index(drop=True)
bi_tri_df_expand["3_gram"] = bi_tri_df_expand["compare"].map(lambda x: x[0] if isinstance(x, tuple) else x)
bi_tri_df_expand["3_gram_count"] = bi_tri_df_expand["compare"].map(lambda x: x[1][0] if isinstance(x, tuple) else x)
bi_tri_df_expand["3_gram_position"] = bi_tri_df_expand["compare"].map(lambda x: x[1][1] if isinstance(x, tuple) else x)
bi_tri_df_expand.reset_index(drop=True)

,2_gram,count,keep_2_gram,compare,group_count,total_related_3_gram,3_gram,3_gram_count,3_gram_position
0,肌膚,15119,True,"(令肌膚, [916, 1.0])",649.0,25336.0,令肌膚,916.0,1.0
1,肌膚,15119,True,"(為肌膚, [767, 1.0])",649.0,25336.0,為肌膚,767.0,1.0
2,肌膚,15119,True,"(肌膚的, [656, 0.0])",649.0,25336.0,肌膚的,656.0,0.0
3,肌膚,15119,True,"(的肌膚, [629, 1.0])",649.0,25336.0,的肌膚,629.0,1.0
4,肌膚,15119,True,"(讓肌膚, [580, 1.0])",649.0,25336.0,讓肌膚,580.0,1.0
...,...,...,...,...,...,...,...,...,...
213828,种天,1,False,NaN,NaN,NaN,NaN,NaN,NaN
213829,种抗,1,False,NaN,NaN,NaN,NaN,NaN,NaN
213830,种有,1,False,NaN,NaN,NaN,NaN,NaN,NaN
213831,种水,1,False,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
bi_tri_df_expand["prob_of_related_3_gram"] = bi_tri_df_expand["3_gram_count"] / bi_tri_df_expand["total_related_3_gram"]
bi_tri_df_expand

,2_gram,count,keep_2_gram,compare,group_count,total_related_3_gram,3_gram,3_gram_count,3_gram_position,prob_of_related_3_gram
0,肌膚,15119,True,"(令肌膚, [916, 1.0])",649.0,25336.0,令肌膚,916.0,1.0,0.036154
0,肌膚,15119,True,"(為肌膚, [767, 1.0])",649.0,25336.0,為肌膚,767.0,1.0,0.030273
0,肌膚,15119,True,"(肌膚的, [656, 0.0])",649.0,25336.0,肌膚的,656.0,0.0,0.025892
0,肌膚,15119,True,"(的肌膚, [629, 1.0])",649.0,25336.0,的肌膚,629.0,1.0,0.024826
0,肌膚,15119,True,"(讓肌膚, [580, 1.0])",649.0,25336.0,讓肌膚,580.0,1.0,0.022892
...,...,...,...,...,...,...,...,...,...,...
149680,种天,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149681,种抗,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149682,种有,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149683,种水,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
bi_tri_df_expand["keep_2_gram"] = bi_tri_df_expand["3_gram_count"] / bi_tri_df_expand["count"] <= 0.9
bi_tri_df_expand["keep_3_gram"] = bi_tri_df_expand["prob_of_related_3_gram"].map(lambda x: True if x > 0.4 else False)

In [26]:
bi_tri_df_expand

,2_gram,count,keep_2_gram,compare,group_count,total_related_3_gram,3_gram,3_gram_count,3_gram_position,prob_of_related_3_gram,keep_3_gram
0,肌膚,15119,True,"(令肌膚, [916, 1.0])",649.0,25336.0,令肌膚,916.0,1.0,0.036154,False
0,肌膚,15119,True,"(為肌膚, [767, 1.0])",649.0,25336.0,為肌膚,767.0,1.0,0.030273,False
0,肌膚,15119,True,"(肌膚的, [656, 0.0])",649.0,25336.0,肌膚的,656.0,0.0,0.025892,False
0,肌膚,15119,True,"(的肌膚, [629, 1.0])",649.0,25336.0,的肌膚,629.0,1.0,0.024826,False
0,肌膚,15119,True,"(讓肌膚, [580, 1.0])",649.0,25336.0,讓肌膚,580.0,1.0,0.022892,False
...,...,...,...,...,...,...,...,...,...,...,...
149680,种天,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
149681,种抗,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
149682,种有,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
149683,种水,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [27]:
retain_2_gram = bi_tri_df_expand[["2_gram", "keep_2_gram"]]
retain_3_gram = bi_tri_df_expand[["3_gram", "keep_3_gram"]]

retain_2_gram.columns = ["n_gram", "retain"]
retain_3_gram.columns = ["n_gram", "retain"]

result_2vs3_df = pd.concat([retain_2_gram, retain_3_gram])

In [28]:
result_2vs3_df.drop_duplicates(inplace=True, subset=["n_gram"])
result_2vs3_df.reset_index(drop=True)

,n_gram,retain
0,肌膚,True
1,配方,True
2,保濕,True
3,使用,True
4,精華,True
...,...,...
213957,淨肌泡,False
213958,淨肌祛,False
213959,淨肌套,False
213960,和淨肌,False


In [29]:
tri_four_df = trigram_df.copy()

count_threshold_3_gram = 150

tri_four_df["keep_3_gram"] = tri_four_df["count"].map(lambda x: True if x > count_threshold_3_gram else False)
tri_four_df

,3_gram,count,keep_3_gram
0,維他命,2812,True
1,屈臣氏,1828,True
2,防腐劑,1193,True
3,型美甲,1179,True
4,美甲片,1124,True
...,...,...,...
343566,杯杯麵,1,False
343567,杯柔滑,1,False
343568,杯榨果,1,False
343569,杯水的,1,False


In [30]:
tri_four_df["compare"] = tri_four_df["3_gram"][tri_four_df["keep_3_gram"] == True].apply(lambda x: find_matches(x, fourgram_df))

In [31]:
tri_four_df

,3_gram,count,keep_3_gram,compare
0,維他命,2812,True,"{'及維他命': [293, 1.0], '含維他命': [210, 1.0], '種維他命..."
1,屈臣氏,1828,True,"{'屈臣氏骨': [253, 0.0], '屈臣氏蒸': [58, 0.0], '屈臣氏燕'..."
2,防腐劑,1193,True,"{'類防腐劑': [179, 1.0], '含防腐劑': [176, 1.0], '加防腐劑..."
3,型美甲,1179,True,"{'型美甲片': [1040, 0.0], '造型美甲': [744, 1.0], '薄型美..."
4,美甲片,1124,True,"{'型美甲片': [1040, 1.0], '繪美甲片': [47, 1.0], '療美甲片..."
...,...,...,...,...
343566,杯杯麵,1,False,NaN
343567,杯柔滑,1,False,NaN
343568,杯榨果,1,False,NaN
343569,杯水的,1,False,NaN


In [32]:
tri_four_df["group_count"] = tri_four_df["compare"].apply(lambda x: len(x.items()) if isinstance(x, dict) else x)
tri_four_df["total_related_4_gram"] = tri_four_df["compare"].apply(lambda x: np.array(list(x.values()))[:, 0].sum() if isinstance(x, dict) and len(x.values()) > 0 else x)
tri_four_df["compare"] = tri_four_df["compare"].map(lambda x: x.items() if isinstance(x, dict) else x)
# tri_four_df[~(tri_four_df["compare"].isna())]
tri_four_df_expand = tri_four_df.explode("compare")  # .groupby("2_gram", as_index=False)  # .agg({"count": "sum"}).sort_values("count", ascending=False).reset_index(drop=True)
tri_four_df_expand["4_gram"] = tri_four_df_expand["compare"].map(lambda x: x[0] if isinstance(x, tuple) else x)
tri_four_df_expand["4_gram_count"] = tri_four_df_expand["compare"].map(lambda x: x[1][0] if isinstance(x, tuple) else x)
tri_four_df_expand["4_gram_position"] = tri_four_df_expand["compare"].map(lambda x: x[1][1] if isinstance(x, tuple) else x)
tri_four_df_expand.reset_index(drop=True)

,3_gram,count,keep_3_gram,compare,group_count,total_related_4_gram,4_gram,4_gram_count,4_gram_position
0,維他命,2812,True,"(及維他命, [293, 1.0])",141.0,2310.0,及維他命,293.0,1.0
1,維他命,2812,True,"(含維他命, [210, 1.0])",141.0,2310.0,含維他命,210.0,1.0
2,維他命,2812,True,"(種維他命, [168, 1.0])",141.0,2310.0,種維他命,168.0,1.0
3,維他命,2812,True,"(維他命及, [137, 0.0])",141.0,2310.0,維他命及,137.0,0.0
4,維他命,2812,True,"(的維他命, [124, 1.0])",141.0,2310.0,的維他命,124.0,1.0
...,...,...,...,...,...,...,...,...,...
355043,杯杯麵,1,False,NaN,NaN,NaN,NaN,NaN,NaN
355044,杯柔滑,1,False,NaN,NaN,NaN,NaN,NaN,NaN
355045,杯榨果,1,False,NaN,NaN,NaN,NaN,NaN,NaN
355046,杯水的,1,False,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
tri_four_df_expand["prob_of_related_4_gram"] = tri_four_df_expand["4_gram_count"] / tri_four_df_expand["total_related_4_gram"]
tri_four_df_expand

,3_gram,count,keep_3_gram,compare,group_count,total_related_4_gram,4_gram,4_gram_count,4_gram_position,prob_of_related_4_gram
0,維他命,2812,True,"(及維他命, [293, 1.0])",141.0,2310.0,及維他命,293.0,1.0,0.12684
0,維他命,2812,True,"(含維他命, [210, 1.0])",141.0,2310.0,含維他命,210.0,1.0,0.090909
0,維他命,2812,True,"(種維他命, [168, 1.0])",141.0,2310.0,種維他命,168.0,1.0,0.072727
0,維他命,2812,True,"(維他命及, [137, 0.0])",141.0,2310.0,維他命及,137.0,0.0,0.059307
0,維他命,2812,True,"(的維他命, [124, 1.0])",141.0,2310.0,的維他命,124.0,1.0,0.05368
...,...,...,...,...,...,...,...,...,...,...
343566,杯杯麵,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343567,杯柔滑,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343568,杯榨果,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343569,杯水的,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
tri_four_df_expand["keep_3_gram"] = tri_four_df_expand["4_gram_count"] / tri_four_df_expand["count"] <= 0.9
tri_four_df_expand["keep_4_gram"] = tri_four_df_expand["prob_of_related_4_gram"].map(lambda x: True if x > 0.4 else False)

In [35]:
tri_four_df_expand

,3_gram,count,keep_3_gram,compare,group_count,total_related_4_gram,4_gram,4_gram_count,4_gram_position,prob_of_related_4_gram,keep_4_gram
0,維他命,2812,True,"(及維他命, [293, 1.0])",141.0,2310.0,及維他命,293.0,1.0,0.12684,False
0,維他命,2812,True,"(含維他命, [210, 1.0])",141.0,2310.0,含維他命,210.0,1.0,0.090909,False
0,維他命,2812,True,"(種維他命, [168, 1.0])",141.0,2310.0,種維他命,168.0,1.0,0.072727,False
0,維他命,2812,True,"(維他命及, [137, 0.0])",141.0,2310.0,維他命及,137.0,0.0,0.059307,False
0,維他命,2812,True,"(的維他命, [124, 1.0])",141.0,2310.0,的維他命,124.0,1.0,0.05368,False
...,...,...,...,...,...,...,...,...,...,...,...
343566,杯杯麵,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
343567,杯柔滑,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
343568,杯榨果,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
343569,杯水的,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [36]:
retain_3_gram = tri_four_df_expand[["3_gram", "keep_3_gram"]]
retain_4_gram = tri_four_df_expand[["4_gram", "keep_4_gram"]]

retain_3_gram.columns = ["n_gram", "retain"]
retain_4_gram.columns = ["n_gram", "retain"]

result_3vs4_df = pd.concat([retain_3_gram, retain_4_gram])

In [37]:
result_3vs4_df.drop_duplicates(inplace=True, subset=["n_gram"])
result_3vs4_df.reset_index(drop=True)

,n_gram,retain
0,維他命,True
1,屈臣氏,True
2,防腐劑,True
3,型美甲,True
4,美甲片,False
...,...,...
355105,膚帶來源,False
355106,膚帶來滿,False
355107,膚帶來緊,False
355108,膚帶來臻,False


In [38]:
four_five_df = fourgram_df.copy()

count_threshold_4_gram = 75

four_five_df["keep_4_gram"] = four_five_df["count"].map(lambda x: True if x > count_threshold_4_gram else False)
four_five_df

,4_gram,count,keep_4_gram
0,透明質酸,1111,True
1,型美甲片,1040,True
2,膠原蛋白,824,True
3,指甲品牌,747,True
4,全新造型,746,True
...,...,...,...
427003,士賦顏黃,1,False
427004,獨家特級,1,False
427005,士豆澱粉,1,False
427006,士護膚產,1,False


In [39]:
four_five_df["compare"] = four_five_df["4_gram"][four_five_df["keep_4_gram"] == True].apply(lambda x: find_matches(x, fivegram_df))

In [40]:
four_five_df

,4_gram,count,keep_4_gram,compare
0,透明質酸,1111,True,"{'透明質酸及': [126, 0.0], '含透明質酸': [98, 1.0], '及透明..."
1,型美甲片,1040,True,"{'造型美甲片': [605, 1.0], '薄型美甲片': [435, 1.0], '型美..."
2,膠原蛋白,824,True,"{'進膠原蛋白': [68, 1.0], '解膠原蛋白': [61, 1.0], '的膠原蛋..."
3,指甲品牌,747,True,"{'指甲品牌全': [744, 0.0], '銷指甲品牌': [741, 1.0], '销指..."
4,全新造型,746,True,"{'全新造型美': [744, 0.0], '牌全新造型': [744, 1.0], '拓全..."
...,...,...,...,...
427003,士賦顏黃,1,False,NaN
427004,獨家特級,1,False,NaN
427005,士豆澱粉,1,False,NaN
427006,士護膚產,1,False,NaN


In [41]:
four_five_df["group_count"] = four_five_df["compare"].apply(lambda x: len(x.items()) if isinstance(x, dict) else x)
four_five_df["total_related_5_gram"] = four_five_df["compare"].apply(lambda x: np.array(list(x.values()))[:, 0].sum() if isinstance(x, dict) and len(x.values()) > 0 else x)
four_five_df["compare"] = four_five_df["compare"].map(lambda x: x.items() if isinstance(x, dict) else x)
# four_five_df[~(four_five_df["compare"].isna())]
four_five_df_expand = four_five_df.explode("compare")  # .groupby("2_gram", as_index=False)  # .agg({"count": "sum"}).sort_values("count", ascending=False).reset_index(drop=True)
four_five_df_expand["5_gram"] = four_five_df_expand["compare"].map(lambda x: x[0] if isinstance(x, tuple) else x)
four_five_df_expand["5_gram_count"] = four_five_df_expand["compare"].map(lambda x: x[1][0] if isinstance(x, tuple) else x)
four_five_df_expand["5_gram_position"] = four_five_df_expand["compare"].map(lambda x: x[1][1] if isinstance(x, tuple) else x)
four_five_df_expand.reset_index(drop=True)

,4_gram,count,keep_4_gram,compare,group_count,total_related_5_gram,5_gram,5_gram_count,5_gram_position
0,透明質酸,1111,True,"(透明質酸及, [126, 0.0])",110.0,1303.0,透明質酸及,126.0,0.0
1,透明質酸,1111,True,"(含透明質酸, [98, 1.0])",110.0,1303.0,含透明質酸,98.0,1.0
2,透明質酸,1111,True,"(及透明質酸, [79, 1.0])",110.0,1303.0,及透明質酸,79.0,1.0
3,透明質酸,1111,True,"(子透明質酸, [68, 1.0])",110.0,1303.0,子透明質酸,68.0,1.0
4,透明質酸,1111,True,"(重透明質酸, [55, 1.0])",110.0,1303.0,重透明質酸,55.0,1.0
...,...,...,...,...,...,...,...,...,...
433409,士賦顏黃,1,False,NaN,NaN,NaN,NaN,NaN,NaN
433410,獨家特級,1,False,NaN,NaN,NaN,NaN,NaN,NaN
433411,士豆澱粉,1,False,NaN,NaN,NaN,NaN,NaN,NaN
433412,士護膚產,1,False,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
four_five_df_expand["prob_of_related_5_gram"] = four_five_df_expand["5_gram_count"] / four_five_df_expand["total_related_5_gram"]
four_five_df_expand

,4_gram,count,keep_4_gram,compare,group_count,total_related_5_gram,5_gram,5_gram_count,5_gram_position,prob_of_related_5_gram
0,透明質酸,1111,True,"(透明質酸及, [126, 0.0])",110.0,1303.0,透明質酸及,126.0,0.0,0.0967
0,透明質酸,1111,True,"(含透明質酸, [98, 1.0])",110.0,1303.0,含透明質酸,98.0,1.0,0.075211
0,透明質酸,1111,True,"(及透明質酸, [79, 1.0])",110.0,1303.0,及透明質酸,79.0,1.0,0.060629
0,透明質酸,1111,True,"(子透明質酸, [68, 1.0])",110.0,1303.0,子透明質酸,68.0,1.0,0.052187
0,透明質酸,1111,True,"(重透明質酸, [55, 1.0])",110.0,1303.0,重透明質酸,55.0,1.0,0.04221
...,...,...,...,...,...,...,...,...,...,...
427003,士賦顏黃,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427004,獨家特級,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427005,士豆澱粉,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427006,士護膚產,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
four_five_df_expand["keep_4_gram"] = four_five_df_expand["5_gram_count"] / four_five_df_expand["count"] <= 0.9

In [44]:
four_five_df_expand

,4_gram,count,keep_4_gram,compare,group_count,total_related_5_gram,5_gram,5_gram_count,5_gram_position,prob_of_related_5_gram
0,透明質酸,1111,True,"(透明質酸及, [126, 0.0])",110.0,1303.0,透明質酸及,126.0,0.0,0.0967
0,透明質酸,1111,True,"(含透明質酸, [98, 1.0])",110.0,1303.0,含透明質酸,98.0,1.0,0.075211
0,透明質酸,1111,True,"(及透明質酸, [79, 1.0])",110.0,1303.0,及透明質酸,79.0,1.0,0.060629
0,透明質酸,1111,True,"(子透明質酸, [68, 1.0])",110.0,1303.0,子透明質酸,68.0,1.0,0.052187
0,透明質酸,1111,True,"(重透明質酸, [55, 1.0])",110.0,1303.0,重透明質酸,55.0,1.0,0.04221
...,...,...,...,...,...,...,...,...,...,...
427003,士賦顏黃,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427004,獨家特級,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427005,士豆澱粉,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427006,士護膚產,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
result_4vs5_df = four_five_df_expand[["4_gram", "keep_4_gram"]]
result_4vs5_df.columns = ["n_gram", "retain"]

In [46]:
result_4vs5_df.drop_duplicates(inplace=True, subset=["n_gram"])
result_4vs5_df.reset_index(drop=True)

C:\Users\victo\AppData\Local\Temp\ipykernel_1124\3885254734.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_4vs5_df.drop_duplicates(inplace=True, subset=["n_gram"])


,n_gram,retain
0,透明質酸,True
1,型美甲片,True
2,膠原蛋白,True
3,指甲品牌,False
4,全新造型,False
...,...,...
427003,士賦顏黃,False
427004,獨家特級,False
427005,士豆澱粉,False
427006,士護膚產,False


In [49]:
summary_df = pd.concat([result_2vs3_df, result_3vs4_df, result_4vs5_df])

In [55]:
summary_df.drop_duplicates(subset=["n_gram"], inplace=True)
summary_df

,n_gram,retain
0,肌膚,True
1,配方,True
2,保濕,True
3,使用,True
4,精華,True
...,...,...
427003,士賦顏黃,False
427004,獨家特級,False
427005,士豆澱粉,False
427006,士護膚產,False


In [56]:
summary_df.to_csv("./n_gram_processed/summary.csv")